# Yacht Insurance Claims Data
##### NOTEBOOK 1

**Problem Statement:** What is the likelihood that a yacht insurance policy has at least 1 claim within five years?

**Contents:**
1. General data cleaning for both datasets.
2. Data cleaning for the master dataset.
3. Data cleaning for the claims dataset.
4. Merging of the two datasets.

<br>**Important to note:** *While cleaning we had to be very careful to keep as much data as possible, especially from theh claims dataset. Before any cleaning our target variable, a policy with at least one claim, was onlly 6.5% of our dataset. Therefore, we were already working with limited data with no way to acquire more.*

___
## Import libraries and read in data

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
# Read in file with only claims
claims = pd.read_excel('../datasets/Claims_report.xlsx')

# Read in file with all the policies from 2016 - 2020
master = pd.read_csv('../datasets/NYP Bordereau Report - Total.xlsx - Bordereau Report.csv')

In [3]:
claims.head()

,Claim No.,Status,Insured name,Policy No.,Policy Eff-Date,Date of Ocurrence,Year,Length,Hull Value,Horsepower,...,Mooring State,Mooring County,Construction,Navigation Limits,Loss,Expense,Loss.1,Expense.1,To Date,Total Incurred
0,1.54715.MA.107,Open,Brent Cox,NYPCAF001878-2,2020-04-04,2020-11-03,1995.0,54.0,580000.0,1640.0,...,AL,BALDWIN,Fiberglass,(D) U.S. Atlantic and Gulf - 1,1000.00,500.0,0.00,0.00,0.0,1500.00
1,1.54181.107.MA,Closed,Brent Cox,NYPCAF001878-2,2020-04-04,2020-10-15,1995.0,54.0,580000.0,1640.0,...,AL,BALDWIN,Fiberglass,(D) U.S. Atlantic and Gulf - 1,618.84,500.0,0.00,0.00,0.0,1118.84
2,1.55475,Open,KEN & HEATHER DEVRIES,NYPCAF001239-3,2020-06-17,2020-10-01,1985.0,44.0,125000.0,60.0,...,NAYARIT,MEXICO,Fiberglass,(A) Pacific - 2,NaN,NaN,NaN,NaN,NaN,NaN
3,1.52983.107MA,Open,STEPHEN & BETH LEATHERS,NYPCAH001805-2,2020-03-05,2020-09-16,2004.0,42.0,300000.0,60.0,...,FL,ESCAMBIA,Fiberglass,(D) U.S. Atlantic and Gulf - 1,0.00,14000.0,310960.00,0.00,0.0,324960.00
4,1.53180.107.MA,Open,Ed Gassman,NYPCAF002704,2019-11-05,2020-08-22,2001.0,45.0,200000.0,900.0,...,FL,BROWARD,Fiberglass,(D) U.S. Atlantic and Gulf - 3,0.00,0.0,25481.25,25481.25,0.0,50962.50


In [4]:
master.head()

,Policy Number,New/Renl/Endt/Canc/Flat,Insurance Broker,First Name,Last Name,Loss Paye,Date of Birth,Married yes/no,Years Exp.,Occupation,...,Hull Type,Hull Limit,Power Type,# Engines,Mooring State,Mooring County,Mooring ZIP code,Credits,Debits,Windstorm Deductible
0,NYPCAF000314,New,Novamar Insurance,Richard,Spindler,Named Insured,05/09/1948,No,2.0,PUBLISHER,...,Multihull Sail,"$ 500,000.00",Inboard,2.0,Nayarit,Mexico,NaN,0%,35%,5%
1,NYPCAF000315,New,Novamar Insurance,Hoc,Tran,United Bank,10/08/1952,NaN,22.0,OWNS SCRAP METAL BUSINESS,...,Sportfisher,"$ 1,275,000.00",Inboard,2.0,CA,Los Angeles,90803,0%,20%,2%
2,NYPCAF000316,Flat,Novamar Insurance,KARL,KRUG,NaN,06/25/1956,Yes,24.0,CEO,...,Trawler,"$ 663,000.00",Inboard,2.0,WASHINGTON,King,98109,0%,0%,2%
3,NYPCAF000316,Flat,Novamar Insurance,KARL,KRUG,NaN,06/25/1956,Yes,24.0,CEO,...,Trawler,"$ 663,000.00",Inboard,2.0,WASHINGTON,King,98109,0%,0%,2%
4,NYPCAF000318,New,Novamar Insurance,JERRY,BORISY,NAMED INSURED,09/30/1943,Yes,30.0,-,...,Motoryacht,"$ 400,000.00",Inboard,2.0,CA,LOS ANGELES,90803,6%,0%,1%


In [5]:
# View the dimensions of each dataset

print('Claims Shape:', claims.shape)
print('Master Shape:', master.shape)

Claims Shape: (410, 25)
Master Shape: (7952, 45)


---
## High level cleaning for both datasets

**BELOW:** In insurance there is a term, "flat". This represents either a new policy or an endorsement on a current policy that was cancelled before it even began. For our purposes we won't be counting them as policies and will drop them immediately.

In [6]:
master.drop(master.loc[(master['New/Renl/Endt/Canc/Flat'] == 'Flat') | 
                       (master['New/Renl/Endt/Canc/Flat'] == 'Endt-Flat')].index, inplace= True)
master.shape

(6320, 45)

**BELOW:** Before any cleaning or exploratory data analysis, of all the policies held from 2016 - 2020, roughly 6.5% had claims.

In [7]:
# Find percentage of policies have claims 
claims.shape[0]  / master.shape[0]

0.06487341772151899

In [8]:
# Lowercase the text in the datasets in case the same words were input with different upper/lowercasing
# Ref: https://stackoverflow.com/questions/39512002/convert-whole-dataframe-from-lower-case-to-upper-case-with-pandas
master = master.applymap(lambda s:s.lower() if type(s) == str else s)
claims = claims.applymap(lambda s:s.lower() if type(s) == str else s)

In [9]:
# Drop any rows that don't have a policy number in both dataframes (this is crucial for us to work with the data)

claims.dropna(subset = ['Policy No.'],inplace=True)
master.dropna(subset = ['Policy Number'], inplace= True)

---

## Cleaning for the Master dataset

In [10]:
# Check for all the null values
master.isna().sum()

Policy Number                   0
New/Renl/Endt/Canc/Flat         0
Insurance Broker                0
First Name                      0
Last Name                       0
Loss Paye                    4456
Date of Birth                   0
Married yes/no               1964
Years Exp.                      0
Occupation                    822
Mailing State                   0
Policy Eff-Date                 0
Endo Eff-Date                5032
Policy/Endo Canc Eff-Date    5620
Hull Prem                       0
Primary Lia Prem                0
Excess Lia Prem                 0
Primary Crew Prem               0
Excess Crew Prem                0
Pol Fee                         0
Total Prem Incl Fee             0
Total Prem Less Fee             0
Collected Premium            5032
Collected Fee                5032
Date Paid                    5032
Collected Total              5032
Unpaid Premium               5032
Unpaid Fee                   5032
Unpaid Total                 5032
Net Prem Due R

### Drop columns we don't need OR have too many nulls

**BELOW:** Columns with the words fee or prem refer to how much the client was charged for a premium. These were dropped because we're interested in features of a boat and person that would lead to the likelihood of having a claim. Additionally, columns that list names of people, insurance broker, or the boat were dropped.

In [11]:
master.drop(columns = ['Loss Paye','First Name', 'Last Name', 'Hull Prem', 'Primary Lia Prem', 'Excess Lia Prem', 'Primary Crew Prem',
       'Excess Crew Prem', 'Pol Fee', 'Total Prem Incl Fee','Insurance Broker',
       'Total Prem Less Fee', 'Collected Premium', 'Collected Fee',
       'Date Paid', 'Collected Total', 'Unpaid Premium', 'Unpaid Fee',
       'Unpaid Total', 'Net Prem Due RFIB', 'Vessel Name', 'Credits', 'Debits', 'Windstorm Deductible'], inplace = True)

In [12]:
master.isna().sum()

Policy Number                   0
New/Renl/Endt/Canc/Flat         0
Date of Birth                   0
Married yes/no               1964
Years Exp.                      0
Occupation                    822
Mailing State                   0
Policy Eff-Date                 0
Endo Eff-Date                5032
Policy/Endo Canc Eff-Date    5620
Builder                         4
Year Built                      0
Construction                   68
Length                          2
Hull Type                       0
Hull Limit                      0
Power Type                      0
# Engines                       0
Mooring State                 118
Mooring County                 28
Mooring ZIP code             1451
dtype: int64

### Fill null Married and Occupation columns

**BELOW:** There were a lot of nulls for each column but we thought they were relevant. Without being able to impute or guess a value to fill them in with, we decided to simply fill them in with 'not reported'.

In [13]:
# Fill both the occupation and married column NAs with 'not reported' for now
master.fillna({'Married yes/no':'not reported', 'Occupation':'not reported'}, inplace=True)

In [14]:
master['Married yes/no'].value_counts()

yes             4106
not reported    1964
no               249
Name: Married yes/no, dtype: int64

### Clean Occupation column further

**BELOW:** As we explored the occupation column further we noticed sometimes a '-' was used in place of a null value. We also replaced this with 'not reported'. Additionally, some values were recorded differently (this was a problem throughout the dataset). For instance, the value self-employed was also recorded as self employed. We changed all values to self employed.

In [15]:
# Check the first 10 top occupations
master['Occupation'].value_counts()[:10]

retired           1402
not reported       822
-                  439
business owner     298
engineer           108
physician           80
attorney            62
ceo                 60
real estate         59
executive           58
Name: Occupation, dtype: int64

In [16]:
# Change '.' and '-' to 'not reported'
master.loc[(master['Occupation'] == '-') | (master['Occupation'] == '.'),'Occupation']='not reported'

# Change 'self-employed' to 'self employed'
master.loc[(master['Occupation'] ==  'self-employed'), 'Occupation']= 'self employed'
master['Occupation'].value_counts()[:20]

retired                 1402
not reported            1273
business owner           298
engineer                 108
self employed             84
physician                 80
attorney                  62
ceo                       60
real estate               59
executive                 58
contractor                54
sales                     52
consultant                51
investor                  43
pilot                     40
cpa                       33
manager                   27
software developer        24
real estate investor      24
architect                 23
Name: Occupation, dtype: int64

**BELOW:** We noticed there were a lot of occupations and decided to only keep the top 20. Otherwise they would be called 'other'.

In [17]:
# Create a list of the top 20 most common occupations
top_20_occ = master['Occupation'].value_counts()[:20].index.to_list()
top_20_occ[:5]

['retired', 'not reported', 'business owner', 'engineer', 'self employed']

In [18]:
# Replace all occupations with 'other' if they aren't in the top 20 most common
master['Occupation'] = master['Occupation'].apply(lambda x: 'other' if x not in top_20_occ else x)
master['Occupation'].value_counts()

other                   2464
retired                 1402
not reported            1273
business owner           298
engineer                 108
self employed             84
physician                 80
attorney                  62
ceo                       60
real estate               59
executive                 58
contractor                54
sales                     52
consultant                51
investor                  43
pilot                     40
cpa                       33
manager                   27
software developer        24
real estate investor      24
architect                 23
Name: Occupation, dtype: int64

### Clean Builder column

**BELOW:** Similar to occupation, we only took the top 20 builders of a boat and changed the remaining to 'other'.

In [19]:
# Create a list of the top 20 most common builders
top_20_build = master['Builder'].value_counts()[:20].index.to_list()

# Replace all the other types with 'other'
master['Builder'] = master['Builder'].apply(lambda x: 'other' if x not in top_20_build else x)
master['Builder'].value_counts()

other                4562
beneteau              298
sea ray               171
island packet         144
lagoon                138
hatteras              137
catalina              118
viking                 95
jeanneau               69
bertram                67
hunter                 60
tiara                  54
cabo                   53
robertson & caine      51
riviera                50
tayana                 45
ocean alexander        44
intrepid               43
offshore               43
catana                 39
carver                 38
Name: Builder, dtype: int64

In [20]:
# Taking a look at the data types for each column
master.dtypes

Policy Number                 object
New/Renl/Endt/Canc/Flat       object
Date of Birth                 object
Married yes/no                object
Years Exp.                   float64
Occupation                    object
Mailing State                 object
Policy Eff-Date               object
Endo Eff-Date                 object
Policy/Endo Canc Eff-Date     object
Builder                       object
Year Built                   float64
Construction                  object
Length                       float64
Hull Type                     object
Hull Limit                    object
Power Type                    object
# Engines                    float64
Mooring State                 object
Mooring County                object
Mooring ZIP code              object
dtype: object

### Clean Date of Birth column

**BELOW:** There was an error converting DOB to datetime object because some of the birth dates were incorrect (born in year 9000). We decided to explore it further and see if we could catch all the incorrect birthdays. After creating a list of 'wrong birthdays' we shared the document with the client and he sent updated birthdays. Then we turned it into a date/time object.

In [21]:
# Create a list of all the DOB years to see where there may be issues
year_list = []
for n in master['Date of Birth']:
    year_list.append(str(n)[6:10])
    
# Sort in descending order and show the top 30
sorted(year_list, reverse=True)[:30]

['9605',
 '9605',
 '9407',
 '9407',
 '2036',
 '2036',
 '2020',
 '2020',
 '2020',
 '2020',
 '2020',
 '2020',
 '2020',
 '2020',
 '2020',
 '2020',
 '2020',
 '2020',
 '2020',
 '2020',
 '2020',
 '2020',
 '2020',
 '2020',
 '1997',
 '1997',
 '1997',
 '1997',
 '1994',
 '1994']

In [22]:
# Create DOB year column
master['DOB_year'] = year_list

# Check the biggest years to see if they make sense
master['DOB_year'].sort_values(ascending=False)

5030    9605
3261    9605
5734    9407
4115    9407
3406    2036
        ... 
495     1932
4351    1931
2620    1931
5895    1931
2251    1928
Name: DOB_year, Length: 6319, dtype: object

In [23]:
# Turn DOB year back to numeric
master['DOB_year'] = pd.to_numeric(master['DOB_year'])

# How many incorrect birthdays?
print(len(master[master['DOB_year'] >= 2020]))

# Create a dataframe to share these errors with the client
wrong_bdays = master[master['DOB_year'] >= 2020]

# Export the dataframe to csv
wrong_bdays.to_csv('../datasets/wrong_bdays.csv')

24


In [24]:
# List of updated birthdays from client
updated_bdays = {'NYPCAH001020-3':'7/25/1957','NYPCAH001020-3':'7/25/1957','NYPCAH001020-3':'7/25/1957',
                 'NYPCAH001020-3':'7/25/1957','NYPCAH001020-3':'7/25/1957','NYPCAH001020-3':'7/25/1957',
                 'NYPCAH001020-3':'7/25/1957','NYPCAH001020-2':'12/14/2020','NYPCAH001020-1':'7/25/1957',
                 'NYPCAH001020':'7/25/1957','NYPCAF002412-1':'1/11/1947','NYPCAF002412':'1/11/1947',
                 'NYPCAF002257-1':'2/16/1936','NYPCAF002257':'2/16/1936','NYPCAF002229-1':'2/21/1965',
                 'NYPCAF002229':'2/21/1965','NYPCAF001453-1':'4/17/1950','NYPCAF001453':'4/17/1950',
                 'NYPCAF000989-3':'9/16/1942','NYPCAF000989-2':'9/16/1942','NYPCAF000989-1':'9/16/1942',
                 'NYPCAF000989':'9/16/1942'}

# Lowercase to match my master df
updated_bdays  = dict((k.lower(), v.lower()) for k,v in updated_bdays.items())

In [25]:
# Create list of keys from dictionary
bday_keys = updated_bdays.keys()

# Create list of values from dictionary
bday_values = updated_bdays.values()

In [26]:
# Grab indices of all policies that need an updated bday
bday_indices = master[master['Policy Number'].isin(bday_keys)].index.to_list()

# Create a new dictionary with indices and new birthdays
bday_ind_dict = dict(zip(bday_indices, bday_values))

# Replace birthdays using dictionary
for key, value in bday_ind_dict.items():
    master.loc[key, 'Date of Birth'] = value

In [27]:
# Update the DOB year column
year_list = []
for n in master['Date of Birth']:
    year_list.append(str(n)[6:10])

# Create DOB year column
master['DOB_year'] = year_list

# Turn into numeric column
master['DOB_year'] = pd.to_numeric(master['DOB_year'])

In [28]:
# Drop all rows where DOB year is greater than 2000
master.drop(master.loc[master['DOB_year'] > 2000].index, inplace=True)

# Turn Date of Birth into date/time object
master['Date of Birth'] = pd.to_datetime(master['Date of Birth'])

In [29]:
master.dtypes

Policy Number                        object
New/Renl/Endt/Canc/Flat              object
Date of Birth                datetime64[ns]
Married yes/no                       object
Years Exp.                          float64
Occupation                           object
Mailing State                        object
Policy Eff-Date                      object
Endo Eff-Date                        object
Policy/Endo Canc Eff-Date            object
Builder                              object
Year Built                          float64
Construction                         object
Length                              float64
Hull Type                            object
Hull Limit                           object
Power Type                           object
# Engines                           float64
Mooring State                        object
Mooring County                       object
Mooring ZIP code                     object
DOB_year                              int64
dtype: object

### Clean Mooring State, County, and ZIP code

**BELOW:** We knew the location of the boat could be very predictive of whether or not a boat may have a claim. However, there were a lot of issues with these columns. County had the least number of nulls so we tried to keep that column and fill in as many as possible. Our steps:<br>
- Some columns had values for zip code but not county. We used the zip to look up the county and fill in values.
- For boats in Mexico, the county and state columns were switched. We switched all of them.
- For any remaining columns, we used the state column for the county column.
- We dropped any remaining rows without a value in county (there were only 2).

In [30]:
# Check relationship between the 3 mooring locations and nulls
master.loc[master['Mooring County'].isna(), ['Policy Number','Mooring County','Mooring State', 'Mooring ZIP code']]

,Policy Number,Mooring County,Mooring State,Mooring ZIP code
25,nypcaf000338,NaN,bcs,23010
277,nypcaf000600,NaN,ca,94965
377,nypcaf000698,NaN,NaN,NaN
395,nypcaf000715,NaN,NaN,NaN
512,nypcaf000832,NaN,hi,96815
597,nypcaf000915,NaN,va,NaN
703,nypcaf001016,NaN,ga,31558
731,nypcaf001044,NaN,bc sur,23010
732,nypcaf001044,NaN,bc sur,23010
851,nypcaf001159,NaN,grenada,NaN


**For any column that had a zip code but no county, we looked them up manually and created a dictionary.**

In [31]:
# Create dictionary by looking up zipcodes manually
mooring_county_dict = {25: 'la paz', 277: 'marin', 512: 'honolulu', 703: 'camden', 731: 'la paz',
                       732: 'la paz', 868: 'bowen island', 1313: 'san mateo', 1339: 'honolulu',
                       1413: 'miami dade',1521: 'tahoka', 1781: 'ensenada', 1867: 'monroe', 
                       2952: 'vancouver', 6281: 'honolulu'}

# Replace counties using dictionary
for key, value in mooring_county_dict.items():
    master.loc[key, 'Mooring County'] = value

In [32]:
# Check that it worked
master['Mooring County'][277]

'marin'

In [33]:
# How many are left
master.loc[master['Mooring County'].isna(), ['Policy Number','Mooring County','Mooring State', 'Mooring ZIP code']]

,Policy Number,Mooring County,Mooring State,Mooring ZIP code
377,nypcaf000698,NaN,NaN,NaN
395,nypcaf000715,NaN,NaN,NaN
597,nypcaf000915,NaN,va,NaN
851,nypcaf001159,NaN,grenada,NaN
1366,nypcaf001467,NaN,bc,vgg3e7
1409,nypcaf001496,NaN,mx,NaN
3085,nypcaf002178,NaN,texas,NaN
4408,nypcaf001982-1,NaN,tobago,NaN
4428,nypcaf002021-1,NaN,sonora,NaN
5947,nypcaf002021-2,NaN,sonora,NaN


**Looks like policies from Mexico have their county/state switched.**

In [34]:
# Show me state and county for all rows where county = mexico 
master.loc[master['Mooring County'] == 'mexico', ['Mooring State', 'Mooring County']]

,Mooring State,Mooring County
0,nayarit,mexico
12,bcs,mexico
15,nayarit,mexico
16,sinaloa,mexico
23,nayarit,mexico
...,...,...
7602,bcs,mexico
7603,bcs,mexico
7610,bcs,mexico
7642,nayarit,mexico


In [35]:
# Switch values for county and mooring state if county = mexico
# Ref: https://stackoverflow.com/questions/25792619/what-is-correct-syntax-to-swap-column-values-for-selected-rows-in-a-pandas-data
master[['Mooring County','Mooring State']] = master[['Mooring State', 'Mooring County']].where(
    master['Mooring County'] == 'mexico', master[['Mooring County','Mooring State']].values)

# Check if these are now in Mooring State
master.loc[(master['Mooring State'] == 'mexico'), ['Mooring State','Mooring County']]

,Mooring State,Mooring County
0,mexico,nayarit
12,mexico,bcs
15,mexico,nayarit
16,mexico,sinaloa
23,mexico,nayarit
...,...,...
7602,mexico,bcs
7603,mexico,bcs
7610,mexico,bcs
7642,mexico,nayarit


In [36]:
master['Mooring State'].value_counts()

fl                    1987
ca                    1420
mexico                 994
wa                     236
tx                     176
                      ... 
south pacifc             1
dominican republic       1
rio dulce                1
great abaco island       1
fronteras                1
Name: Mooring State, Length: 135, dtype: int64

In [37]:
master['Mooring County'].value_counts()

broward          394
bcs              373
los angeles      353
orange           339
san diego        334
                ... 
hillsborough       1
honolulu           1
guererro           1
hampton            1
orange county      1
Name: Mooring County, Length: 254, dtype: int64

In [38]:
master['Mooring County'].fillna(master['Mooring State'],  inplace=True)
master['Mooring County'].isna().sum()

2

In [39]:
# Drop remaining rows where Mooring County is still null
master.dropna(subset= ['Mooring County'], inplace=True)

### Drop remaining rows in the 'master' dataframe with very few null values.

In [40]:
master.isna().sum()

Policy Number                   0
New/Renl/Endt/Canc/Flat         0
Date of Birth                   0
Married yes/no                  0
Years Exp.                      0
Occupation                      0
Mailing State                   0
Policy Eff-Date                 0
Endo Eff-Date                5023
Policy/Endo Canc Eff-Date    5609
Builder                         0
Year Built                      0
Construction                   68
Length                          2
Hull Type                       0
Hull Limit                      0
Power Type                      0
# Engines                       0
Mooring State                 110
Mooring County                  0
Mooring ZIP code             1445
DOB_year                        0
dtype: int64

In [41]:
master.dropna(subset= ['Length'], inplace = True)

---

## Cleaning for the Claims dataset

In [42]:
claims.head(2)

,Claim No.,Status,Insured name,Policy No.,Policy Eff-Date,Date of Ocurrence,Year,Length,Hull Value,Horsepower,...,Mooring State,Mooring County,Construction,Navigation Limits,Loss,Expense,Loss.1,Expense.1,To Date,Total Incurred
0,1.54715.ma.107,open,brent cox,nypcaf001878-2,2020-04-04,2020-11-03,1995.0,54.0,580000.0,1640.0,...,al,baldwin,fiberglass,(d) u.s. atlantic and gulf - 1,1000.00,500.0,0.0,0.0,0.0,1500.00
1,1.54181.107.ma,closed,brent cox,nypcaf001878-2,2020-04-04,2020-10-15,1995.0,54.0,580000.0,1640.0,...,al,baldwin,fiberglass,(d) u.s. atlantic and gulf - 1,618.84,500.0,0.0,0.0,0.0,1118.84


**BELOW:** We decided to drop columns that would become redundant when merged with the master dataframe (like 'Policy Eff-Date'), unnecessary to our problem (like 'Total Incurred'), or would not have matching values from the master dataframe and therefore cause too many nulls in the merged dataset (like 'Navigation Limits').

In [43]:
# Drop columns that won't be necessary for the model
claims.drop(columns= ['Insured name', 'Policy Eff-Date', 'Power Type', 'Navigation Limits', 'Loss', 'Expense', 
                      'Loss.1', 'Expense.1', 'To Date', 'Total Incurred'], inplace = True)

### Check which remaining columns are null and fill them in

In [44]:
claims.isna().sum()

Claim No.               0
Status                  0
Policy No.              0
Date of Ocurrence       0
Year                    0
Length                  0
Hull Value              0
Horsepower              0
Accident description    0
Manufacturer            0
Hull Type               0
Fuel Type               0
Mooring State           5
Mooring County          1
Construction            3
dtype: int64

### Fill in construction column

In [45]:
claims[claims['Construction'].isna()]

,Claim No.,Status,Policy No.,Date of Ocurrence,Year,Length,Hull Value,Horsepower,Accident description,Manufacturer,Hull Type,Fuel Type,Mooring State,Mooring County,Construction
23,1.51611.107.ma,open,nypcaf001839-2,2020-05-19,2012.0,46.0,390000.0,108.0,lightning/electrical surge,robertson & caine - leopard,multihull sail,diesel,bcn,mexico,NaN
104,1.45950.107.ma,closed,nypcaf001839,2019-07-23,2012.0,46.0,390000.0,108.0,lightning/electrical surge,robertson & caine - leopard,multihull sail,diesel,fl,charlotte,NaN
176,1.42814.107.ma,closed,nypcaf001788,2018-11-25,2002.0,42.0,130000.0,56.0,theft / vandalism,beneteau,monohull sail,diesel,hi,honolulu,NaN


**BELOW:** Got the construction type for these boats with claims from the client.

In [46]:
# Update the main df with the correct construction types for boats with claims
claims.loc[23, 'Construction'] = 'Fiberglass'
claims.loc[104, 'Construction'] = 'Fiberglass'
claims.loc[176, 'Construction'] = 'Fiberglass'

### Fill in county column

In [47]:
# Look at all the null states compared to their counties
claims.loc[claims['Mooring State'].isna(), ['Mooring State', 'Mooring County']]

,Mooring State,Mooring County
158,NaN,med
213,NaN,med
262,NaN,panama
280,NaN,panama
370,NaN,NaN


In [48]:
# Fill null states with county values
claims['Mooring State'].fillna(claims['Mooring County'],  inplace=True)
claims.loc[claims['Mooring State'].isna(), ['Mooring State', 'Mooring County']]

,Mooring State,Mooring County
370,NaN,NaN


In [49]:
claims['Mooring State'].fillna('other', inplace=True)

claims['Mooring County'].fillna('other', inplace=True)

claims.isna().sum()

Claim No.               0
Status                  0
Policy No.              0
Date of Ocurrence       0
Year                    0
Length                  0
Hull Value              0
Horsepower              0
Accident description    0
Manufacturer            0
Hull Type               0
Fuel Type               0
Mooring State           0
Mooring County          0
Construction            0
dtype: int64

### Create a column with number of claims per unique policy.

In [50]:
# Look at all policies with more than one claim

repeat_pol_df = claims[claims.duplicated(subset= ['Policy No.'], keep=False)]
print(len(repeat_pol_df['Policy No.'].value_counts()))
repeat_pol_df['Policy No.'].value_counts()

31


nypcah001308-2    3
nypcaf001167-1    3
nypcaf002224      3
nypcaf001715      3
nypcaf000980      2
nypcaf001330      2
nypcaf001878-2    2
nypcah001315      2
nypcaf001777      2
nypcaf000622      2
nypcaf002064-1    2
nypcaf000555      2
nypcaf001531      2
nypcaf000339      2
nypcaf001288      2
nypcah002283      2
nypcaf002143-1    2
nypcaf002689      2
nypcaf001197      2
nypcaf000625      2
nypcaf000872      2
nypcaf000676-1    2
nypcaf000588      2
nypcaf001180-1    2
nypcaf000902      2
nypcaf002323      2
nypcaf002068      2
nypcaf000651      2
nypcaf001415      2
nypcaf001430-1    2
nypcaf000930      2
Name: Policy No., dtype: int64

In [51]:
# Set the above series to a variable
series = repeat_pol_df['Policy No.'].value_counts()

# Turn the series into a dictionary
repeat_claims_dict = series.to_dict() 

# Create number of claims column per policy number
# Ref https://stackoverflow.com/questions/51881503/python-pandas-assign-a-dictionary-value-to-a-dataframe-column-based-on-dictionar
claims['num_claims'] = claims['Policy No.'].apply(lambda x: repeat_claims_dict.get(x))

claims['num_claims'].value_counts()

2.0    54
3.0    12
Name: num_claims, dtype: int64

In [52]:
# Fill na columns with 1 for having only 1 claim
claims['num_claims'] = claims['num_claims'].fillna(1)

claims['num_claims'].value_counts()

1.0    343
2.0     54
3.0     12
Name: num_claims, dtype: int64

In [53]:
claims.columns

Index(['Claim No.', 'Status', 'Policy No.', 'Date of Ocurrence', 'Year',
       'Length', 'Hull Value', 'Horsepower', 'Accident description',
       'Manufacturer', 'Hull Type', 'Fuel Type', 'Mooring State',
       'Mooring County', 'Construction', 'num_claims'],
      dtype='object')

In [54]:
master.columns

Index(['Policy Number', 'New/Renl/Endt/Canc/Flat', 'Date of Birth',
       'Married yes/no', 'Years Exp.', 'Occupation', 'Mailing State',
       'Policy Eff-Date', 'Endo Eff-Date', 'Policy/Endo Canc Eff-Date',
       'Builder', 'Year Built', 'Construction', 'Length', 'Hull Type',
       'Hull Limit', 'Power Type', '# Engines', 'Mooring State',
       'Mooring County', 'Mooring ZIP code', 'DOB_year'],
      dtype='object')

In [55]:
# Now drop any columns that will be repeats from the master df
claims.drop(columns= ['Length', 'Manufacturer', 'Mooring State', 'Mooring County', 'Construction', 
                      'Hull Value', 'Horsepower', 'Accident description','Manufacturer', 'Hull Type',
                      'Fuel Type', 'Claim No.','Status', 'Date of Ocurrence', 'Year'], inplace = True)

---

## Merge dataframes

In [56]:
# Rename column we will use to merge
claims = claims.rename(columns={'Policy No.':'Policy Number'})

In [57]:
combined = pd.merge(master, claims, on='Policy Number', how='left')
combined.head()

,Policy Number,New/Renl/Endt/Canc/Flat,Date of Birth,Married yes/no,Years Exp.,Occupation,Mailing State,Policy Eff-Date,Endo Eff-Date,Policy/Endo Canc Eff-Date,...,Length,Hull Type,Hull Limit,Power Type,# Engines,Mooring State,Mooring County,Mooring ZIP code,DOB_year,num_claims
0,nypcaf000314,new,1948-05-09,no,2.0,other,california,03/31/2016,NaN,NaN,...,63.0,multihull sail,"$ 500,000.00",inboard,2.0,mexico,nayarit,NaN,1948,NaN
1,nypcaf000315,new,1952-10-08,not reported,22.0,other,california,03/18/2016,NaN,NaN,...,61.0,sportfisher,"$ 1,275,000.00",inboard,2.0,ca,los angeles,90803,1952,NaN
2,nypcaf000318,new,1943-09-30,yes,30.0,not reported,california,03/30/2016,NaN,NaN,...,48.0,motoryacht,"$ 400,000.00",inboard,2.0,ca,los angeles,90803,1943,NaN
3,nypcaf000319,new,1977-04-25,not reported,20.0,not reported,california,03/30/2016,NaN,NaN,...,32.0,motoryacht,"$ 35,000.00",inboard,0.0,ca,los angeles,90802,1977,NaN
4,nypcaf000320,new,1951-06-28,no,30.0,not reported,california,04/02/2016,NaN,NaN,...,43.0,monohull sail,"$ 200,000.00",inboard,1.0,ca,los angeles,90803,1951,NaN


## Save and export dataframe

In [58]:
combined.to_csv('../datasets/combined.csv', index= False)

---
## Conclusion:

In this notebook we cleaned both datasets separately with 3 main things in mind:<br>
1. Keep as much data as possible, especially from the claims dataset.
2. What columns are necessary to clean and retain for the eventual merge of the two datasets.
3. What columns are most important for predicting whether or not a boat will have claims.

## Next Notebook:

In the following notebook, *02_Model_Preprocessing*, we will get the now merged dataframe ready for modeling. We will do this through a combination of cleaning, feature engineering, and exploratory data analysis to understand whch features to keep or drop.